In [ ]:
# Copyright 2018 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Construct a Model for Computer Vision using Keras

This *ML pipeline' constructs a Convolutional Neural Network (CNN) using Keras framework, as follows:

        1. Configurable # of 2D Convolutional Layer, with configurable input size.
        2. Max Pooling and Flattening Layer.
        3. Configurable # of Neural Network layers, with configurable number of nodes.
        4. Configurable # of dropout Layer, with configurable percentage.
        5. Output Layer, with configurable number of outputs (classes).

In [ ]:
# Keras's Neural Network components
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

# Kera's Convoluntional Neural Network components
from keras.layers import Conv2D, MaxPooling2D

## Setting Parameters

Use the `construct_cnn()` routine to construct the CNN, which is construct as:

                    Convolutional -> Neural Network -> Output

The `input_size` is the (height,width) shape of the preprocessed image data (machine learning ready data). For example, in the MNIST and EMMNIST datasets, the input size is (28,28).

The `n_classes` is the number of classes to train the model for. Each class is a distinct object to recognize (e.g., a cat). The number of classes will be the number of nodes in the output layer of the neural network. For example, in the MNIST and EMMNIST datasets, the number of classes is 10 and 62 respectively.

The `n_nodes` may either be a single integer value or a list of integer values. When specified as a single integer, the value is the number of nodes in the input layer of the neural network from the convolutional front-end, and there are no hidden layers.

Otherwise, `n_nodes` is a list, the first list element is the number of nodes in the input layer of the neural network from the convolutional front-end. The remaining elements are the hidden layers, where the value of the element is the number of nodes in the corresponding hidden layer.

The `dropout` is the percentage of dropout after the first layer of the neural network. If the value is 0, then there is no dropout.


In [ ]:
def construct_cnn(input_shape, n_classes, n_filters=32, n_nodes=128, dropout=0):
    """ Construct a CNN for Training and Inference.
    Args:
        input_shape: (tuple(int,int,int)) the 3D shape of the input vector.
        n_classes  : (int) total number of classes.
        n_filters  : (tuple(int,...)) number of filters per convolutional layer
        n_nodes    : (tuple(int,...)) number of nodes per neural network layer.
        dropout    : (float) Dropout rate between 0 and 1.
    
    Returns:
        A compiled model.

    Raises:
        None.
    """
    # Constructing a Feed Forward Neural Network
    model = Sequential()
    
    # make n_filters a tuple if a single int
    if isinstance(n_filters, int):
        n_filters = tuple([n_filters])

    # Add a first convolutional front-end with 3x3 kernal
    model.add(Conv2D(filters=n_filters[0], kernel_size=3, padding='same', activation='relu', input_shape=input_shape))

    # Add Remaining Convolutional layers
    for ix in range(1, len(n_filters)):
        if n_filters[ix] == True:
            # Add max pooling layer
            model.add(MaxPooling2D(pool_size=2))
        else:
            # Add next convolutional front-end with 3x3 kernal
            model.add(Conv2D(filters=n_filters[ix], kernel_size=3, padding='same', activation='relu'))     
    
    # Add max pooling layer
    model.add(MaxPooling2D(pool_size=2))
              
    # Flatten the output from the max pooling layer for input to the neural network
    model.add(Flatten())
    
    # make n_nodes a tuple if a single integer
    if isinstance(n_nodes, int):
        n_nodes = tuple([n_nodes])
        
    # make dropout a tuple if a single integer
    if isinstance(dropout, int) or isinstance(dropout, float):
        # apply dropout to the first layer
        dropout = [dropout]
        # make remaining layers zero
        for _ in range(1, len(n_nodes)):
            dropout.append(0)
        dropout = tuple(dropout)
        
    # Add layers
    for ix in range(len(n_nodes)):
        model.add(Dense(n_nodes[ix], activation='relu'))
        # Add dropout if any
        if dropout[ix] > 0:
            model.add(Dropout(dropout[ix]))

    # Add the output layer
    model.add(Dense(n_classes, activation='softmax'))
    
    # Choice the loss function and optimizer and finish construcing the CNN
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
    return model


In the function train_cnn() there are two ways of passing the train/test data.

1. Unsplit: the train and test data are passed as combined data, in which case x_test and y_test are None. The function will then shuffle the combined data and then split the combined data into training and test based on the percent parameter.

2. Split: the train and test data are already split, in which case x and y are the training data and x_test and y_test are the test data. The function does not shuffle or split the pre-split data, and the percent parameter is ignored.

In [ ]:
import numpy as np
from keras.utils import np_utils

def train_cnn(model, x, y, x_test=None, y_test=None, epochs=10, batch_size=32, percent=0.2, verbose=False, seed=113):
    """ Train the model
    Args:
        model     : (model) The CNN model.
        x         : (numpy.ndarray) The x portion (preprocessed image data) of the dataset.
        y         : (numpy.ndarray) The y portion (labels) of the dataset.
        x_test    : (numpy.ndarray) The x_test (if pre-split) portion of the dataset.
        y_test    : (numpy.ndarray) The y_test (if pre-split) portion of the dataset.
        epochs    : (int) The number of times to feed the entire dataset for training.
        batch_size: (int) The mini-batch size.
        percent   : (float) The percent of the dataset to use for test.
        verbose   : (bool) Display (console) progress status.
        seed      : (int) Seed for random shuffle before splitting.
        learning_rate: (float) The learning rate.
        
    Returns:
        The model accuracy after training and evaluation.
    
    Raises:
        None
    """
    
    # one hot encode the labels
    y = np_utils.to_categorical(y)
    if y_test is not None:
        y_test = np_utils.to_categorical(y_test)
    
    
    # Images are grayscale. Keras expects shape to be (rows, height, width, channels) vs. (rows, height, width)
    if len(x.shape) == 3:
        x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
        if x_test is not None:
            x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)     
            
    # Ignore percent if data is already split
    if x_test is not None:
        percent = 0
    
    
    # Calculate the number of elements which will be used as training data
    train_size = int((1-percent) * len(x))
    if verbose: print("Training Size:", train_size)
     
    # Dataset is combined
    if x_test is None:
        # Randomly shuffle the data before splitting
        np.random.seed(seed)
        np.random.shuffle(x)
        np.random.seed(seed)
        np.random.shuffle(y)

        # split the data into Train and Test
        X_train = x[:train_size]
        Y_train = y[:train_size]
        X_test  = x[train_size:]
        Y_test  = y[train_size:]
    # Dataset is presplit
    else:
        X_train = x
        Y_train = y
        X_test  = x_test
        Y_test  = y_test

    if verbose: verbose = 2
        
    # Train the model with the training data
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size, shuffle=True, verbose=verbose)
    
    # Run the model with the test (evaluation) data
    accuracy = model.evaluate(X_test, Y_test, verbose=0)
    
    return accuracy

    

In [ ]:
def codeGenerator(input_shape, n_classes, n_filters=32, n_nodes=128, dropout=0):
    code = []
    code.append("# Keras's Neural Network components")
    code.append("from keras.models import Sequential")
    code.append("from keras.layers import Dense, Dropout, Activation, Flatten")
    code.append("# Kera's Convoluntional Neural Network components")
    code.append("from keras.layers import Conv2D, MaxPooling2D")
    code.append("")
    code.append("def construct_cnn():")
    code.append("\t# Constructing a Feed Forward Neural Network")
    code.append("\tmodel = Sequential()")
    code.append("")
    
    # make n_filters a tuple if a single int
    if isinstance(n_filters, int):
        n_filters = tuple([n_filters])
        
    # Add a first convolutional front-end with 3x3 kernal
    code.append("\tmodel.add(Conv2D(filters=" + str(n_filters[0]) + ", kernel_size=3, padding='same', activation='relu', input_shape=" + str(input_shape) + "))")

    # Add Remaining Convolutional layers
    for ix in range(1, len(n_filters)):
        if n_filters[ix] == True:
            # Add max pooling layer
            code.append("\tmodel.add(MaxPooling2D(pool_size=2))")
        else:
            # Add next convolutional front-end with 3x3 kernal
            code.append("\tmodel.add(Conv2D(filters=" + str(n_filters[ix]) + ", kernel_size=3, padding='same', activation='relu'))")   
                
    # Add max pooling layer
    code.append("\tmodel.add(MaxPooling2D(pool_size=2))")
              
    # Flatten the output from the max pooling layer for input to the neural network
    code.append("\tmodel.add(Flatten())")
    
    # make n_nodes a tuple if a single integer
    if isinstance(n_nodes, int):
        n_nodes = tuple([n_nodes])
        
    # make dropout a tuple if a single integer
    if isinstance(dropout, int) or isinstance(dropout, float):
        # apply dropout to the first layer
        dropout = [dropout]
        # make remaining layers zero
        for _ in range(1, len(n_nodes)):
            dropout.append(0)
        dropout = tuple(dropout)
        
    # Add layers
    for ix in range(len(n_nodes)):
        code.append("\tmodel.add(Dense(" + str(n_nodes[ix]) + ", activation='relu'))")
        # Add dropout if any
        if dropout[ix] > 0:
            code.append("\tmodel.add(Dropout(" + str(dropout[ix]) + "))")     

    # Add the output layer
    code.append("\tmodel.add(Dense(" + str(n_classes) + ", activation='softmax'))")
    
    # Choice the loss function and optimizer and finish construcing the CNN
    code.append("\tmodel.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])")
    
    code.append("\treturn model")
    for line in code:
        print(line)


In [ ]:
#print("Simple Covnet")
#codeGenerator((28,28,1), 10, n_filters=(32, True, 64))

#print("VGG16")
#codeGenerator( (224, 224, 3), 1000, n_filters=(64, 64, True, 128, 128, True, 256, 256, 256, True, 512, 512, 512, True, 512, 512, 512), n_nodes=(4096, 4096))